# Install - Import libraries

In [ ]:
# installation of three libraries: plotly, nbformat, and colour-science

%pip install -q plotly nbformat colour-science ipywidgets seaborn

In [ ]:
# import the necessary libraries

import numpy as np                                # package for numerical computation
import matplotlib                                 # package for plotting
import matplotlib.pyplot as plt                   # package for plotting
import plotly.express as px                       # package for interactive plotting
from plotly.subplots import make_subplots         # import a subplot feature from the plotly library
from glob import glob                             # package to manipulate files
import pandas as pd                               # package to manipulate data in table
from sklearn.metrics import pairwise_distances    # package for mathematical computation
import os                                         # package for dealing with operating system environment
import colour                                     # package for colorimetry
import nbformat
import seaborn as sns

from ipywidgets import Layout, Button, Box, interact, interactive, fixed, interact_manual
import ipywidgets as wg
from IPython.display import display, clear_output

In [ ]:
cd /home/gus/Documents/RCE/projects/2021-141_SdG_MBVB-Kusama/data/processed/RS/

In [ ]:
cd ../data/RS/processed/

# Object - Measurement spots

In total, 64 measurements were performed on 16 distinct locations on the object and referred to as *groups*. Four measurements were taken for each group for which the mean and standard deviation have been calculated.

<img src="../figures/Kusuma_BEK-1858-P_NL_groups.png" width="700px">

$\Large{\text{Figure 1. Kusuma, Infinity Mirror Room - Phalli’s Field, details, (BEK 1858 P)}}$

# Groups - Summary table

In [ ]:
dic_spec = {
    "1": "D",
    "2": "B",
    "3": "C",
    "4": "D",
    "5": "B",
    "6": "C",
    "7a": "A",
    "7b": "A",
    "7c": "A",
    "8a": "A",
    "8b": "A",
}
dic_cover = {
    "01": "no",
    "02": "no",
    "03": "no",
    "04": "no",
    "05": "no",
    "06": "no",
    "07": "yes",
    "08": "yes",
    "09": "no",
    "10": "no",
    "11": "yes",
    "12": "yes",
    "13": "no",
    "14": "yes",
    "15": "no",
    "16": "yes",
}
files = sorted(glob("*txt*"))

groups = [
    pd.read_csv(file, index_col="parameter").loc["group"]["value_mean"][-2:]
    for file in files
]
patroons = [
    pd.read_csv(file, index_col="parameter")
    .loc["comments"]["value_mean"]
    .split("_")[0][7:]
    for file in files
]
colours = [
    pd.read_csv(file, index_col="parameter").loc["color"]["value_mean"]
    for file in files
]

df_groups = pd.DataFrame(
    data={
        "groups": groups,
        "patroons": patroons,
        "spectra": [dic_spec[patroon] for patroon in patroons],
        "colour": colours,
        "covered": [dic_cover[group] for group in groups],
    }
).set_index("groups")

df_sorted = df_groups.sort_values(by=["colour", "spectra"])

In [ ]:
df_sorted

# Tab viewer 

In [ ]:
files = sorted(glob("*txt*"))
style = {"description_width": "initial"}
path_interim = '/drive/data/RS/processed/'

device = 'KM, CM-2600d'
filter = 'none'
observer = '10°'
illuminant = 'D65'
d65 = colour.CCS_ILLUMINANTS["cie_10_1964"]["D65"]

def tab_viewer():
    
    projects_list = ['2021-140','2021-141','2021-142']
    analyses_Id_list = [f'{file.split("_")[1]}_{file.split("_")[2]}_{file.split("_")[3]}' for file in files]
    projects = wg.SelectMultiple(options=projects_list, value=['2021-140'], rows=4, description='Project Id', style=style)
    sample_nb = wg.Dropdown(options = analyses_Id_list, description='Analyses Id', style=style)
    wg_search_button = wg.Button(description='Search')
    output_button = wg.Output()
    output_plot = wg.Output()
    
    
    
    def change_RS(change):
        output_plot.clear_output(wait = True)
        
        with output_plot:
            Id = change.new
            file  = path_interim + glob(f'*{Id}*')[0]            
            df = pd.read_csv(file, index_col='parameter')            
            
            df_sp = df.loc['400':,:]
            
            wl = df_sp.index.astype(float)            
            sp_mean = df_sp['value_mean'].astype(float).values
            sp_std = df_sp['value_std'].astype(float).values 
            
                       
            
            gs_kw = dict(width_ratios=[10,3], height_ratios=[1])
            fig, [ax1,ax2] = plt.subplots(1,2,figsize = (13,6), gridspec_kw = gs_kw)
            sns.set()
            fs = 20             
                                              
            
            L_mean = float(df.loc['L*']['value_mean'])
            a_mean = float(df.loc['a*']['value_mean'])
            b_mean = float(df.loc['b*']['value_mean'])
            C_mean = df.loc['C*']['value_mean']
            h_mean = df.loc['h']['value_mean'] 
            
            L_std = df.loc['L*']['value_std']
            a_std = df.loc['a*']['value_std']
            b_std = df.loc['b*']['value_std']
            C_std = df.loc['C*']['value_std']
            h_std = df.loc['h']['value_std']
                        
            Lab_mean = [L_mean,a_mean,b_mean]            
            srgb = colour.XYZ_to_sRGB(colour.Lab_to_XYZ(Lab_mean), d65).clip(0, 1)
                        
            
            ax1.plot(wl,sp_mean)
            ax1.fill_between(wl, sp_mean+sp_std, sp_mean-sp_std, alpha=0.3, color='0.3')

            cp = matplotlib.patches.Rectangle((0,0), 1, 0.45, color=srgb)
            ax2.add_patch(cp)

            text_ax1 = '\n'.join((     
            f'Device : {device}',
            f'Filter : {filter}',     
            ))


            text_ax2 = '\n'.join((
            f'{illuminant} - {observer}',
            r' ',
            r'$L^*$ = '+ str(Lab_mean[0]) + ' +/- ' + str(L_std),
            r'$a^*$ = '+ str(Lab_mean[1]) + ' +/- ' + str(a_std),
            r'$b^*$ = '+ str(Lab_mean[2]) + ' +/- ' + str(b_std),
            r'$C^*$ = '+ C_mean + ' +/- ' + str(C_std),
            r'$h$ = '+ h_mean + ' +/- ' + str(h_std),
            ))

            props_ax1 = dict(boxstyle='round', facecolor='white', alpha=0.6, edgecolor=None)
            props_ax2 = dict(boxstyle='round', facecolor='0.89', alpha=0.6, edgecolor=None)

            # place a text box in upper left in axes coords
            ax1.text(0.02,0.96,text_ax1,transform=ax1.transAxes,fontsize=fs-6,verticalalignment='top', bbox=props_ax1)
            ax2.text(0.04,0.98,text_ax2,transform=ax2.transAxes,fontsize=fs-4,verticalalignment='top', bbox=props_ax2)

            ax1.set_xlim(wl[0],wl[-1])

            ax1.set_xlabel('Wavelength $\lambda$ (nm)', fontsize=fs)
            ax1.set_ylabel('Reflectance factor', fontsize=fs)
            #ax1.set_title(f'{date}, {project_Id}, {Id}, {group}', fontsize=fontsize)

            ax1.xaxis.set_tick_params(labelsize=fs)
            ax1.yaxis.set_tick_params(labelsize=fs)

            fig.patch.set_facecolor('0.94')

            ax2.axis('off')
            
            plt.tight_layout()
            plt.show()
            
        
        
    
    def search_button(b):

        with output_button:

            clear_output()
            display(sample_nb)
    
    
    wg_search_button.on_click(search_button)

    
    sample_nb.observe(change_RS, names = 'value')
    
    display(wg.VBox([projects,wg_search_button,output_button, output_plot]))
    
tab_viewer()


# Spectra 
## Plots

In [ ]:
df_RS_reds = pd.DataFrame()

for file in files:
    df = pd.read_csv(file, index_col='parameter')
    color = df.loc['color']['value_mean']
    
    if color == 'red':
    
        group = df.loc['group']['value_mean'][-2:]
        patroon = df_groups.loc[group]['patroons']
        spectrum = df_groups.loc[group]['spectra']
        
        data_sp = df.loc['[MEASUREMENT DATA]':][6:].astype('float')

        wl = data_sp.index.astype('float')
        sp_mean = data_sp['value_mean']
        sp_std = data_sp['value_std']
        
        df_data = pd.DataFrame(data = {'groups':group,'spectra':spectrum, 'wl':wl.values, 'sp':sp_mean.values})
        df_RS_reds = pd.concat([df_RS_reds,df_data], axis=0)
        
        
fig = px.line(df_RS_reds, x ='wl',y='sp', color='groups') 

fig.update_layout(
    autosize=False,
    width=1200,
    height=600,
    title="BEK 1858 P, red dots",
    xaxis_title="Wavelength (nm)",
    yaxis_title="Reflectance factor",
    legend_title="Groups",
    font=dict(
        family="Droid Serif",
        size=22,
        color="Black"
    )
)

fig.show()  

In [ ]:
df_RS_whites = pd.DataFrame()

for file in files:
    df = pd.read_csv(file, index_col='parameter')
    color = df.loc['color']['value_mean']
    
    if color == 'white':
    
        group = df.loc['group']['value_mean'][-2:]
        patroon = df_groups.loc[group]['patroons']
        spectrum = df_groups.loc[group]['spectra']
        
        data_sp = df.loc['[MEASUREMENT DATA]':][6:].astype('float')

        wl = data_sp.index.astype('float')
        sp_mean = data_sp['value_mean']
        sp_std = data_sp['value_std']
        
        df_data = pd.DataFrame(data = {'groups':group,'spectra':spectrum, 'wl':wl.values, 'sp':sp_mean.values})
        df_RS_whites = pd.concat([df_RS_whites,df_data], axis=0)
        
        
fig = px.line(df_RS_whites, x ='wl',y='sp', color='groups') 

fig.update_layout(
    autosize=False,
    width=1200,
    height=600,
    title="BEK 1858 P, white textile",
    xaxis_title="Wavelength (nm)",
    yaxis_title="Reflectance factor",
    legend_title="Groups",
    font=dict(
        family="Droid Serif",
        size=22,
        color="Black"
    )
)

fig.show() 

In [ ]:
fig = px.line(df_RS_reds.sort_values(by=['spectra','groups','wl']), x ='wl',y='sp', line_group='groups', color='spectra')

fig.update_layout(
    autosize=False,
    width=1200,
    height=600,
    title="BEK 1858 P, red dots",
    xaxis_title="Wavelength (nm)",
    yaxis_title="Reflectance factor",
    legend_title="Spectra",
    font=dict(
        family="Droid Serif",
        size=22,
        color="Black"
    )
)

fig.show()

## Pairwise distance 

In [ ]:
df_grps = pd.DataFrame(df_RS_reds.groups.unique())
spectra = (df_RS_reds.sp.values.reshape(11, -1))
spderiv = np.diff(spectra, axis = 1)

reordered_grps = [0,7, 8, 9, 10, 1, 2, 3, 4, 5, 6]
reordered_df = df_grps.iloc[reordered_grps,:]
reordered_sp = spectra[reordered_grps, :]
reordered_spderiv = spderiv[reordered_grps, :]

fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(20,10))
fs = 22

ax1.imshow(pairwise_distances(spectra, metric='cosine'))
ax2.imshow(pairwise_distances(reordered_sp, metric='cosine'))
ax3.imshow(pairwise_distances(reordered_spderiv, metric='cosine'))

ax1.set_xticks(np.arange(0,11))
ax1.set_xticklabels(df_grps.iloc[:,0].values, fontsize=fs)

ax1.set_yticks(np.arange(0,11))
ax1.set_yticklabels(df_grps.iloc[:,0].values, fontsize=fs)

ax1.set_xlabel('Groups', fontsize=fs)
ax1.set_ylabel('Groups', fontsize=fs)

ax1.set_title('Non clustered groups', fontsize=fs+2)



ax2.set_xticks(np.arange(0,11))
ax2.set_xticklabels(reordered_df.iloc[:,0].values, fontsize=fs)

ax2.set_yticks(np.arange(0,11))
ax2.set_yticklabels(reordered_df.iloc[:,0].values, fontsize=fs)

ax2.set_xlabel('Groups', fontsize=fs)
ax2.set_ylabel('Groups', fontsize=fs)

ax2.set_title('Clustered groups', fontsize=fs+2)


ax3.set_xticks(np.arange(0,11))
ax3.set_xticklabels(reordered_df.iloc[:,0].values, fontsize=fs)

ax3.set_yticks(np.arange(0,11))
ax3.set_yticklabels(reordered_df.iloc[:,0].values, fontsize=fs)

ax3.set_xlabel('Groups', fontsize=fs)
ax3.set_ylabel('Groups', fontsize=fs)

ax3.set_title('1st derivative - Clustered groups', fontsize=fs+2)


ax2.annotate('D', xy=(0.9,1.1), color='white', weight='bold', fontsize=fs)
ax2.annotate('B', xy=(6.85,7.15), color='white', weight='bold', fontsize=fs)
ax2.annotate('C', xy=(9.3,9.7), color='white', weight='bold', fontsize=fs)
ax2.annotate('A', xy=(3.8,4.1), color='white', weight='bold', fontsize=fs)

ax3.annotate('D', xy=(0.9,1.1), color='white', weight='bold', fontsize=fs)
ax3.annotate('B', xy=(6.85,7.15), color='white', weight='bold', fontsize=fs)
ax3.annotate('C', xy=(9.3,9.7), color='white', weight='bold', fontsize=fs)
ax3.annotate('A', xy=(3.8,4.1), color='white', weight='bold', fontsize=fs)

plt.tight_layout()
plt.show()

# Reproducibility - Group 1 

In [ ]:
file_mean = sorted(glob('*group01*'))[0]
df_mean = pd.read_csv(file_mean, index_col='parameter')
Lab_mean = df_mean.loc[['L*','a*','b*']]['value_mean'].astype(float).values

folder_RS_interim = '/home/gus/Documents/RCE/projects/2021-141_SdG_MBVB-Kusama/data/interim/RS/'
files = [file for file in os.listdir(folder_RS_interim) if 'group01' in file]

dE00s = []

for file in files:
    df = pd.read_csv(folder_RS_interim + file, index_col='parameter')
    Lab = df.loc[['L*','a*','b*']]['value1'].astype(float).values
    dE00 = colour.delta_E(Lab_mean,Lab)
    dE00s.append(dE00)

MCDM = np.round(np.mean(dE00s),2)

print(f'The mean colour difference to the mean (MCDM) has a value of {MCDM} dE00.')
print('This means that colour differences smaller the 0.33 cannot be interpretated as changes within materials.')

# Repeatability - Group 2 

In [ ]:
file_mean = sorted(glob('*group02*'))[0]
df_mean = pd.read_csv(file_mean, index_col='parameter')
Lab_mean = df_mean.loc[['L*','a*','b*']]['value_mean'].astype(float).values

folder_RS_interim = '/home/gus/Documents/RCE/projects/2021-141_SdG_MBVB-Kusama/data/interim/RS/'
files = [file for file in os.listdir(folder_RS_interim) if 'group02' in file]

dE00s = []

for file in files:
    df = pd.read_csv(folder_RS_interim + file, index_col='parameter')
    Lab = df.loc[['L*','a*','b*']]['value1'].astype(float).values
    dE00 = colour.delta_E(Lab_mean,Lab)
    dE00s.append(dE00)

MCDM = np.round(np.mean(dE00s),2)

print(f'The MCDM for the repeatability is {MCDM} dE00.')
print('This is a bit less interesting in our case. Reproducibility is more meaningful here.')

# CIELAB 

In [ ]:
files = sorted(glob("*txt*"))
fmt = {"A": "o", "B": "X", "C": "s", "D": ">"}
colors = {
    "01": "LightSeaGreen",
    "02": "red",
    "03": "blue",
    "04": "yellow",
    "05": "black",
    "06": "aqua",
    "07": "darkslategray",
    "08": "darkred",
    "09": "darksalmon",
    "10": "dodgerblue",
    "11": "forestgreen",
    "12": "magenta",
    "13": "olive",
    "14": "orchid",
    "15": "palegreen",
    "16": "orange",
}

fig = make_subplots(rows=2,
                    cols=2,     
                    )

fs = 20
cps = 6  # capsize values

df_Lab = pd.DataFrame()

for file in files:
    df = pd.read_csv(file, index_col="parameter")
    color = df.loc["color"]["value_mean"]

    if color == "red":

        group = df.loc["group"]["value_mean"][-2:]
        patroon = df_groups.loc[group]["patroons"]
        spectrum = df_groups.loc[group]["spectra"]

        data_Lab = df.loc[["L*", "a*", "b*"]].astype(float)

        L_mean = np.round(data_Lab.loc["L*"]["value_mean"], 2)
        L_std = np.round(data_Lab.loc["L*"]["value_std"], 3)

        a_mean = np.round(data_Lab.loc["a*"]["value_mean"], 2)
        a_std = np.round(data_Lab.loc["a*"]["value_std"], 3)

        b_mean = np.round(data_Lab.loc["b*"]["value_mean"], 2)
        b_std = np.round(data_Lab.loc["b*"]["value_std"], 3)

        fig.add_scatter(
            x=[L_mean],
            y=[b_mean],
            #error_x=dict(value=L_std),
            #error_y=dict(value=b_std),
            legendgroup=group,
            mode="markers",
            name=group,
            marker=dict(size=5, color=colors[group]),
            row=1,
            col=1,
        )

        fig.add_scatter(
            x=[a_mean],
            y=[b_mean],
            #error_x=dict(value=a_std),
            #error_y=dict(value=b_std),
            legendgroup=group,
            mode="markers",
            name=group,
            showlegend=False,
            marker=dict(size=5, color=colors[group]),
            row=1,
            col=2,
        )

        fig.add_scatter(
            x=[a_mean],
            y=[L_mean],
            #error_x=dict(value=a_std),
            #error_y=dict(value=L_std),
            legendgroup=group,
            mode="markers",
            name=group,
            showlegend=False,
            marker=dict(size=5, color=colors[group]),
            row=2,
            col=2,
        )


fig.update_layout(
    autosize=False,
    width=700,
    height=700,
    title="BEK 1858 P, red dots",
    legend_title="Groups",
    font=dict(family="Droid Serif", size=17, color="Black"),
    margin=dict(l=30, r=30, b=30, t=50, pad=2),
    paper_bgcolor="white",
)


layout = {
    "xaxis": {
        "title": r"CIE $L^*$",        
    },

    "yaxis": {
        "title": r"CIE $b^*$"
    },  
    
    "xaxis2":{
        "title": r"CIE $a^*$"
    }, 

    "xaxis4":{
        "title": r"CIE $a^*$"
    },  

    "yaxis4":{
        "title": r"CIE $L^*$"
    },  
}

fig.update_layout(layout)
fig.show()

In [ ]:
files = sorted(glob("*txt*"))
fmt = {"A": "o", "B": "X", "C": "s", "D": ">"}
colors = {
    "A": "LightSeaGreen",
    "B": "red",
    "C": "blue",
    "D": "forestgreen",
}

fig = make_subplots(rows=2, cols=2)

fs = 20
cps = 6  # capsize values

df_Lab = pd.DataFrame()

for file in files:
    df = pd.read_csv(file, index_col="parameter")
    color = df.loc["color"]["value_mean"]

    if color == "red":

        group = df.loc["group"]["value_mean"][-2:]
        patroon = df_groups.loc[group]["patroons"]
        spectrum = df_groups.loc[group]["spectra"]

        data_Lab = df.loc[["L*", "a*", "b*"]].astype(float)

        L_mean = np.round(data_Lab.loc["L*"]["value_mean"], 2)
        L_std = np.round(data_Lab.loc["L*"]["value_std"], 2)

        a_mean = np.round(data_Lab.loc["a*"]["value_mean"], 2)
        a_std = np.round(data_Lab.loc["a*"]["value_std"], 2)

        b_mean = np.round(data_Lab.loc["b*"]["value_mean"], 2)
        b_std = np.round(data_Lab.loc["b*"]["value_std"], 2)

        fig.add_scatter(
            x=[L_mean],
            y=[b_mean],
            error_x=dict(value=L_std),
            error_y=dict(value=b_std),
            legendgroup=spectrum,
            mode="markers",
            name=f"{spectrum}, {group}",
            marker=dict(size=7, color=colors[spectrum]),
            row=1,
            col=1,
        )

        fig.add_scatter(
            x=[a_mean],
            y=[b_mean],
            error_x=dict(value=a_std),
            error_y=dict(value=b_std),
            legendgroup=spectrum,
            mode="markers",
            name=f"{spectrum}, {group}",
            showlegend=False,
            marker=dict(size=7, color=colors[spectrum]),
            row=1,
            col=2,
        )

        fig.add_scatter(
            x=[a_mean],
            y=[L_mean],
            error_x=dict(value=a_std),
            error_y=dict(value=L_std),
            legendgroup=spectrum,
            mode="markers",
            name=f"{spectrum}, {group}",
            showlegend=False,
            marker=dict(size=7, color=colors[spectrum]),
            row=2,
            col=2,
        )


fig.update_layout(
    autosize=False,
    width=700,
    height=700,
    title="BEK 1858 P, red dots",
    legend_title="Spectrum, Group",
    font=dict(family="Droid Serif", size=17, color="Black"),
    margin=dict(l=30, r=30, b=30, t=40, pad=2),
    paper_bgcolor="white",
)

layout = {
    "xaxis": {
        "title": r"CIE $L^*$",
    },
    "yaxis": {"title": r"CIE $b^*$"},
    "xaxis4": {"title": r"CIE $a^*$"},
    "yaxis4": {"title": r"CIE $L^*$"},
}


fig.update_layout(layout)
fig.show()